In [1]:
# Import libraries
import os
import cv2
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 0=ALL, 1=INFO, 2=WARNING, 3=ERROR
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
# load model
model=load_model('../models/trained_model/v_model.keras')
print('Model Loaded!')
#model.summary()

Model Loaded!


In [4]:
# Define a list of colors for visualization
colors = [(245,117,16), (117,245,16), (16,117,245),(200,103,27),(117,245,16)]

# Function to visualize probabilities on the frame
def prob_viz(pred_probs, labels, frame, colors=colors):
    output_frame = frame.copy()
    for num, prob in enumerate(pred_probs[0]):
        cv2.rectangle(output_frame, (0,60+num*30), (int(prob*100), 90+num*30), colors[num], -1)
        cv2.putText(output_frame, labels[num], (0, 85+num*30), 
                    cv2.FONT_HERSHEY_COMPLEX_SMALL    , 0.8, (255,255,255), 1, cv2.LINE_AA)
        
    return output_frame

In [5]:
# Prediction Function
def prediction_on_video(model, path, window_size=3, step_size=3, interval=5, n_frames=18):

    # Map the predicted class to the corresponding shot category based on your label encoding
    labels = ['Bowled', 'Cover Drive', 'Defence', 'Pull Shot', 'Reverse Sweep']

    records_pred = []
    pred_probs = [[0, 0, 0, 0, 0]]
    pred_shot = ""

    # Open a video capture object
    cap = cv2.VideoCapture(path)  # Replace with the path to your video file

    # Create a video writer object for saving the predicted result
    video = cv2.VideoWriter('predicted_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20, (840, 480))

    start_time = 0
    end = 0

    while True:
        # Set the video capture to the start time
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_time * cap.get(cv2.CAP_PROP_FPS)))

        frames = []
        frame_counter = 0

        while len(frames) < n_frames:
            success, frame = cap.read()

            if not success:
                end = 1
                break

            frame = cv2.resize(frame, (840, 480))
            out_frame = cv2.resize(frame, (840, 480))
            last_frame = cv2.resize(frame, (840, 480))

            if frame_counter % interval == 0:
                frame = cv2.resize(frame, (199, 199))
                frames.append(frame)

            frame_counter += 1

            out_frame = prob_viz(pred_probs, labels, out_frame)
            cv2.rectangle(out_frame, (0, 0), (840, 30), (245, 117, 16), -1)
            cv2.putText(out_frame, ' '.join(records_pred[-4:]), (3, 25),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

            video.write(out_frame)

            cv2.imshow('Cricket PoseNet : AI Shot Assistant', out_frame)
            cv2.waitKey(20)

        # Pad or trim frames to have a consistent length of max_frames
        frames = frames + [np.zeros_like(frames[0])] * max(0, n_frames - len(frames))

        # Prediction
        pred_probs = model.predict(np.array([frames]), verbose=0)
        pred_class = np.argmax(pred_probs, axis=1)
        pred_shot = labels[pred_class[0]]
        records_pred.append(pred_shot)

        if end:
            out_frame = prob_viz(pred_probs, labels, last_frame)
            cv2.rectangle(out_frame, (0, 0), (840, 30), (245, 117, 16), -1)
            cv2.putText(out_frame, ' '.join(records_pred[-4:]), (3, 25),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

            cv2.imshow('Cricket PoseNet : AI Shot Assistant', out_frame)
            cv2.waitKey(2000)
            for i in range(10):
                video.write(out_frame)
            break

        # Update the time for the next window
        start_time += step_size

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object
    video.release()
    cap.release()
    cv2.destroyAllWindows()

    return records_pred

In [6]:
# Take prediction using webcam by  set path=0 or prediction on video by set path = "folder/video.mp4"  
prediction_on_video(model,path="./test1.mp4")

['Cover Drive', 'Defence', 'Pull Shot', 'Reverse Sweep']

In [64]:
# Prediction Function
def prediction_on_video2(model, path, window_size=3, step_size=3, interval=5, n_frames=18):

    # Map the predicted class to the corresponding shot category based on your label encoding
    labels = ['Bowled', 'Cover Drive', 'Defence', 'Pull', 'Reverse Sweep']
    records =[]
    confidence=[]
    records_pred_class = []
    records_pred = []
    pred_probs = [[0, 0, 0, 0, 0]]
    pred_shot = ""

    # Open a video capture object
    cap = cv2.VideoCapture(path)  # Replace with the path to your video file

    # Create a video writer object for saving the predicted result
    video = cv2.VideoWriter('predicted_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20, (840, 480))

    start_time = 0
    end = 0

    while True:
        # Set the video capture to the start time
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_time * cap.get(cv2.CAP_PROP_FPS)))

        frames = []
        frame_counter = 0

        while len(frames) < n_frames:
            success, frame = cap.read()

            if not success:
                end = 1
                break

            frame = cv2.resize(frame, (840, 480))
            out_frame = cv2.resize(frame, (840, 480))
            last_frame = cv2.resize(frame, (840, 480))

            if frame_counter % interval == 0:
                frame = cv2.resize(frame, (199, 199))
                frames.append(frame)

            frame_counter += 1

            out_frame = prob_viz(pred_probs, labels, out_frame)
            cv2.rectangle(out_frame, (0, 0), (840, 30), (245, 117, 16), -1)
            cv2.putText(out_frame, ' '.join(records_pred[-4:]), (3, 25),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

            video.write(out_frame)

            cv2.imshow('Cricket PoseNet : AI Shot Assistant', out_frame)
            cv2.waitKey(2)

        # Pad or trim frames to have a consistent length of max_frames
        frames = frames + [np.zeros_like(frames[0])] * max(0, n_frames - len(frames))

        # Prediction
        pred_probs = model.predict(np.array([frames]), verbose=0)
        records.append(pred_probs[0])
        pred_class = np.argmax(pred_probs, axis=1)
        records_pred_class.append(pred_class[0])
        pred_shot = labels[pred_class[0]]
        records_pred.append(pred_shot)
        conf = float(np.max(pred_probs[0]))*100
        confidence.append(conf)

        if end:
            out_frame = prob_viz(pred_probs, labels, last_frame)
            cv2.rectangle(out_frame, (0, 0), (840, 30), (245, 117, 16), -1)
            cv2.putText(out_frame, ' '.join(records_pred[-4:]), (3, 25),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

            cv2.imshow('Cricket PoseNet : AI Shot Assistant', out_frame)
            cv2.waitKey(2000)
            for i in range(10):
                video.write(out_frame)
            break

        # Update the time for the next window
        start_time += step_size

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object
    video.release()
    cap.release()
    cv2.destroyAllWindows()

    print(records)
    print(confidence)
    print(records_pred_class)

    # Initialize an empty dictionary to store the counts
    # Initialize the dictionary with 0 for each key
    json = {
        'shots_played': 0,
        'shot_sequence': 0,
        'Bowled_count': 0,
        'Cover Drive_count': 0,
        'Defence_count': 0,
        'Pull_count': 0,
        'Reverse Sweep_count': 0,
        'Bowled_avg_prob': 0,
        'Cover Drive_avg_prob': 0,
        'Defence_avg_prob': 0,
        'Pull_avg_prob': 0,
        'Reverse Sweep_avg_prob': 0,
        'Bowled_shot_runs': 0,
        'Cover Drive_shot_runs': 0,
        'Defence_shot_runs': 0,
        'Pull_shot_runs': 0,
        'Reverse Sweep_shot_runs': 0,
        'good_shot': "",
        'bad_shot': ""
    }
    # shot_counts = {}
    json['shots_played'] = len(records_pred)
    json['shot_sequence'] = records_pred
    # Count the number of each shot
    for i,shot in enumerate(records_pred):
        if f'{shot}_count' in json:
            json[f'{shot}_count'] += 1
            json[f'{shot}_avg_prob']+= round(confidence[i], 2)
    for shot in labels:
        if json[f'{shot}_count'] !=0 :
            json[f'{shot}_avg_prob'] = round((json[f'{shot}_avg_prob']/json[f'{shot}_count']),2)

    

    count = json['Cover Drive_count'] + json['Defence_count']//3 + json['Pull_count'] +json['Reverse Sweep_count']
    json['Cover Drive_shot_runs'] = round((json['Cover Drive_count']/count)*100,2)
    json['Defence_shot_runs'] = round(((json['Defence_count']//3)/count)*100,2)
    json['Pull_shot_runs'] = round((json['Pull_count']/count)*100,2)
    json['Reverse Sweep_shot_runs'] = round((json['Reverse Sweep_count']/count)*100,2)

    json['good_shot'] = labels[np.argmax([0, json["Cover Drive_avg_prob"], json["Defence_avg_prob"], json["Pull_avg_prob"], json["Reverse Sweep_avg_prob"]])]
    json['bad_shot'] = labels[np.argmin([100-json["Bowled_avg_prob"], json["Cover Drive_avg_prob"], json["Defence_avg_prob"], json["Pull_avg_prob"], json["Reverse Sweep_avg_prob"]])]
    tf.print(json)

    return records_pred

In [65]:
# Take prediction using webcam by  set path=0 or prediction on video by set path = "folder/video.mp4"  
prediction_on_video2(model,path="./test2.mp4")

[array([0.04048261, 0.8924748 , 0.00151025, 0.01455937, 0.05097308],
      dtype=float32), array([0.04085872, 0.8907802 , 0.00146585, 0.01478648, 0.05210875],
      dtype=float32), array([1.9634758e-04, 3.5562008e-04, 2.2579703e-05, 9.9917775e-01,
       2.4768992e-04], dtype=float32), array([0.8610067 , 0.01374909, 0.00545731, 0.11184951, 0.00793754],
      dtype=float32), array([0.44882974, 0.3378361 , 0.00654918, 0.17780754, 0.02897749],
      dtype=float32), array([0.29635075, 0.00863033, 0.67070484, 0.01930751, 0.00500654],
      dtype=float32), array([1.87104568e-04, 8.35377781e-04, 1.43996385e-05, 9.98437583e-01,
       5.25516749e-04], dtype=float32), array([2.5917580e-02, 6.4942775e-05, 4.9260834e-06, 3.1455289e-05,
       9.7398108e-01], dtype=float32)]
[89.24747705459595, 89.07802104949951, 99.91777539253235, 86.10066771507263, 44.88297402858734, 67.07048416137695, 99.84375834465027, 97.39810824394226]
[1, 1, 3, 0, 0, 2, 3, 4]
{'Bowled_avg_prob': 65.49,
 'Bowled_count': 2,
 

['Cover Drive',
 'Cover Drive',
 'Pull',
 'Bowled',
 'Bowled',
 'Defence',
 'Pull',
 'Reverse Sweep']

In [74]:
# Prediction Function
def prediction_on_video3(model, path, window_size=3, step_size=3, interval=5, n_frames=18):

    # Map the predicted class to the corresponding shot category based on your label encoding
    labels = ['Bowled', 'Cover Drive', 'Defence', 'Pull', 'Reverse Sweep']
    records =[]
    confidence=[]
    records_pred_class = []
    records_pred = []
    pred_probs = [[0, 0, 0, 0, 0]]
    pred_shot = ""

    # Open a video capture object
    cap = cv2.VideoCapture(path)  # Replace with the path to your video file

    # Create a video writer object for saving the predicted result
    video = cv2.VideoWriter('predicted_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20, (840, 480))

    start_time = 0
    end = 0

    while True:
        # Set the video capture to the start time
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_time * cap.get(cv2.CAP_PROP_FPS)))

        frames = []
        frame_counter = 0

        while len(frames) < n_frames:
            success, frame = cap.read()

            if not success:
                end = 1
                break

            frame = cv2.resize(frame, (840, 480))
            out_frame = cv2.resize(frame, (840, 480))
            last_frame = cv2.resize(frame, (840, 480))

            if frame_counter % interval == 0:
                frame = cv2.resize(frame, (199, 199))
                frames.append(frame)

            frame_counter += 1

            out_frame = prob_viz(pred_probs, labels, out_frame)
            cv2.rectangle(out_frame, (0, 0), (840, 30), (245, 117, 16), -1)
            cv2.putText(out_frame, ' '.join(records_pred[-4:]), (3, 25),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

            video.write(out_frame)

            cv2.imshow('Cricket PoseNet : AI Shot Assistant', out_frame)
            cv2.waitKey(2)

        # Pad or trim frames to have a consistent length of max_frames
        frames = frames + [np.zeros_like(frames[0])] * max(0, n_frames - len(frames))

        # Prediction
        pred_probs = model.predict(np.array([frames]), verbose=0)
        records.append(pred_probs[0])
        pred_class = np.argmax(pred_probs, axis=1)
        records_pred_class.append(pred_class[0])
        pred_shot = labels[pred_class[0]]
        records_pred.append(pred_shot)
        conf = float(np.max(pred_probs[0]))*100
        confidence.append(conf)

        if end:
            out_frame = prob_viz(pred_probs, labels, last_frame)
            cv2.rectangle(out_frame, (0, 0), (840, 30), (245, 117, 16), -1)
            cv2.putText(out_frame, ' '.join(records_pred[-4:]), (3, 25),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

            cv2.imshow('Cricket PoseNet : AI Shot Assistant', out_frame)
            cv2.waitKey(2000)
            for i in range(10):
                video.write(out_frame)
            break

        # Update the time for the next window
        start_time += step_size

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object
    video.release()
    cap.release()
    cv2.destroyAllWindows()

    # print(records)
    # print(confidence)
    # print(records_pred_class)

    # Initialize an empty dictionary to store the counts
    # Initialize the dictionary with 0 for each key
    json = {
        'shots_played': 0,
        'shot_sequence': 0,
        'Bowled_count': 0,
        'Cover Drive_count': 0,
        'Defence_count': 0,
        'Pull_count': 0,
        'Reverse Sweep_count': 0,
        'Bowled_avg_prob': 0,
        'Cover Drive_avg_prob': 0,
        'Defence_avg_prob': 0,
        'Pull_avg_prob': 0,
        'Reverse Sweep_avg_prob': 0,
        'Bowled_shot_runs': 0,
        'Cover Drive_shot_runs': 0,
        'Defence_shot_runs': 0,
        'Pull_shot_runs': 0,
        'Reverse Sweep_shot_runs': 0,
        'better_shot': "",
        'weak_shot': ""
    }
    # shot_counts = {}
    json['shots_played'] = len(records_pred)
    json['shot_sequence'] = records_pred
    # Count the number of each shot
    for i,shot in enumerate(records_pred):
        if f'{shot}_count' in json:
            json[f'{shot}_count'] += 1
            json[f'{shot}_avg_prob']+= round(confidence[i], 2)
    for shot in labels:
        if json[f'{shot}_count'] !=0 :
            json[f'{shot}_avg_prob'] = round((json[f'{shot}_avg_prob']/json[f'{shot}_count']),2)

    

    count = json['Cover Drive_count'] + json['Defence_count']//3 + json['Pull_count'] +json['Reverse Sweep_count']
    json['Cover Drive_shot_runs'] = round((json['Cover Drive_count']/count)*100,2)
    json['Defence_shot_runs'] = round(((json['Defence_count']//3)/count)*100,2)
    json['Pull_shot_runs'] = round((json['Pull_count']/count)*100,2)
    json['Reverse Sweep_shot_runs'] = round((json['Reverse Sweep_count']/count)*100,2)

    json['better_shot'] = labels[np.argmax([0, json["Cover Drive_avg_prob"], json["Defence_avg_prob"], json["Pull_avg_prob"], json["Reverse Sweep_avg_prob"]])]
    json['weak_shot'] = labels[np.argmin([100-json["Bowled_avg_prob"], json["Cover Drive_avg_prob"], json["Defence_avg_prob"], json["Pull_avg_prob"], json["Reverse Sweep_avg_prob"]])]
    # tf.print(json)

    return json

In [75]:
# Take prediction using webcam by  set path=0 or prediction on video by set path = "folder/video.mp4"  
prediction_on_video3(model,path="./test2.mp4")

{'shots_played': 8,
 'shot_sequence': ['Cover Drive',
  'Cover Drive',
  'Pull',
  'Bowled',
  'Bowled',
  'Defence',
  'Pull',
  'Reverse Sweep'],
 'Bowled_count': 2,
 'Cover Drive_count': 2,
 'Defence_count': 1,
 'Pull_count': 2,
 'Reverse Sweep_count': 1,
 'Bowled_avg_prob': 65.49,
 'Cover Drive_avg_prob': 89.16,
 'Defence_avg_prob': 67.07,
 'Pull_avg_prob': 99.88,
 'Reverse Sweep_avg_prob': 97.4,
 'Bowled_shot_runs': 0,
 'Cover Drive_shot_runs': 40.0,
 'Defence_shot_runs': 0.0,
 'Pull_shot_runs': 40.0,
 'Reverse Sweep_shot_runs': 20.0,
 'better_shot': 'Pull',
 'weak_shot': 'Bowled'}